## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/movielens_ratings.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
# Create a view or table

temp_table_name = "movielens_ratings_csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `movielens_ratings_csv`

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "movielens_ratings_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
import pyspark

In [0]:
data=spark.read.csv("/FileStore/tables/movielens_ratings.csv",inferSchema=True,header=True)

In [0]:
data.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 2| 3.0| 0|
 3| 1.0| 0|
 5| 2.0| 0|
 9| 4.0| 0|
 11| 1.0| 0|
 12| 2.0| 0|
 15| 1.0| 0|
 17| 1.0| 0|
 19| 1.0| 0|
 21| 1.0| 0|
 23| 1.0| 0|
 26| 3.0| 0|
 27| 1.0| 0|
 28| 1.0| 0|
 29| 1.0| 0|
 30| 1.0| 0|
 31| 1.0| 0|
 34| 1.0| 0|
 37| 1.0| 0|
 41| 2.0| 0|
+-------+------+------+
only showing top 20 rows

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
train_data,test_data=data.randomSplit([0.8,0.2])

In [0]:
als=ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [0]:
model=als.fit(train_data)

In [0]:
predictions=model.transform(test_data)

In [0]:
predictions.show()

+-------+------+------+----------+
movieId|rating|userId|prediction|
+-------+------+------+----------+
 31| 1.0| 4| 2.2431095|
 31| 1.0| 18| -2.411983|
 31| 2.0| 25| 2.8898082|
 31| 1.0| 29| 2.4239779|
 85| 1.0| 5| 1.6458768|
 85| 5.0| 16| 2.4238307|
 85| 2.0| 20|-2.7339365|
 85| 1.0| 26| 3.8335323|
 85| 1.0| 28| 3.8549833|
 65| 1.0| 4|0.15922241|
 65| 1.0| 16| 2.0605264|
 65| 5.0| 23| 2.904427|
 53| 5.0| 8| 3.360095|
 53| 1.0| 12| 0.7732836|
 78| 1.0| 4| 1.2937337|
 78| 1.0| 13| 1.1589609|
 34| 1.0| 0| 2.4163034|
 34| 1.0| 4|0.65164816|
 34| 1.0| 16| 2.5706463|
 34| 3.0| 25| 1.395978|
+-------+------+------+----------+
only showing top 20 rows

In [0]:
eval=RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [0]:
rmse=eval.evaluate(predictions)

In [0]:
rmse

Out[12]: 2.0378233644244585

In [0]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId','userId'])

In [0]:
single_user.show()

+-------+------+
movieId|userId|
+-------+------+
 11| 11|
 16| 11|
 22| 11|
 25| 11|
 30| 11|
 35| 11|
 39| 11|
 47| 11|
 50| 11|
 61| 11|
 90| 11|
+-------+------+

In [0]:
reccomendations = model.transform(single_user)

In [0]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
movieId|userId| prediction|
+-------+------+-----------+
 22| 11| 3.847561|
 90| 11| 2.487048|
 30| 11| 2.4037402|
 25| 11| 1.9720728|
 35| 11| 0.8375809|
 16| 11| 0.7702201|
 61| 11| 0.6335211|
 11| 11| 0.20444849|
 50| 11|-0.64192355|
 39| 11| -0.6736163|
 47| 11| -0.6761824|
+-------+------+-----------+